# Capstone Project - The Battle of the Neighborhoods

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for an Indoor multi-sports arena. Specifically, this report will be targeted to stakeholders interested in opening an **Recreational multi-sport arena** in **Toronto**, Canada.

Since there is a population boom in toronto and recreational play is preferred by people of all ages,  we will try to detect **locations that do not have many indoor sports arenas**. We are also particularly interested in **areas with less gyms in vicinity**. We would also prefer locations **as close to populous centers like Downtown Toronto as possible**, assuming that first two conditions are met.

We try to find few most promissing neighborhoods based on our data science analysis.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing sports arena in the neighborhood (any type of arena).
* number of and distance to gyms in the neighborhood, if any.
* distance of neighborhood from city center.


Following data sources will be needed to extract/generate the required information:
* number of sports arena and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Toronto center will be obtained using **Geopy geocoding** of well known Toronto location (Downtown Totonto)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 6x6 killometers centered around Downtown Toronto city center.

Let's first find the latitude & longitude of Downtown Toronto city center, using specific, well known address and Geopy geocoding API.

In [80]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import traceback
import pickle
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math
print('Libraries imported.')

Libraries imported.


In [81]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.653963, -79.387207.


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Downtown Toronto. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [82]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Toronto center longitude={}, latitude={}'.format(longitude, latitude))
x, y = lonlat_to_xy(longitude, latitude)
print('Toronto center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Toronto center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Toronto center longitude=-79.387207, latitude=43.653963
Toronto center UTM X=-5310408.823314987, Y=10507908.004696788
Toronto center longitude=-79.38720700000049, latitude=43.65396299999976


Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors**.

In [83]:
toronto_center_x, toronto_center_y = lonlat_to_xy(longitude, latitude) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = toronto_center_x - 6000
x_step = 600
y_min = toronto_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(toronto_center_x, toronto_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [84]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.Marker([latitude, longitude], popup='DownTown').add_to(map_toronto)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_toronto)    
map_toronto

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from DownTown Toronto. 

Let's now use Geopy API to get approximate addresses of those locations.

In [93]:
def get_address(latitude, longitude, verbose=False):
    try:
        g = geolocator.geocode([latitude, longitude])
        return g
    except:
        return None
    
addr =  get_address(latitude, longitude)
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(latitude, longitude, addr))

Reverse geocoding check
-----------------------
Address of [43.653963, -79.387207] is: None


In [88]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    addres = get_address(lat, lon)
    address = str(addres)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Canada', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [89]:
addresses[150:170]

['1, Highland Gardens, Rosedale, University—Rosedale, Old Toronto, Toronto, Golden Horseshoe, Ontario, M4T 1E6',
 'Our Lady of Perpetual Help Catholic School, 1, Mount Pleasant Road, Moore Park, University—Rosedale, Old Toronto, Toronto, Golden Horseshoe, Ontario, M4T 1E6',
 '253, St. Clair Avenue East, Moore Park, University—Rosedale, Old Toronto, Toronto, Golden Horseshoe, Ontario, M4T 1E6',
 'Avenue Of The Islands, Harbourfront, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5J 2E9',
 'Toronto Island BBQ & Beer Company, 61, Avenue Of The Islands, Harbourfront, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5J 2E9',
 'Toronto Island BBQ & Beer Company, 61, Avenue Of The Islands, Harbourfront, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5J 2E9',
 'None',
 'None',
 'None',
 'South Core, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5E',
 'TD South Tower, 79, Wellington Street West, Toronto-Domin

Looking good. Let's now place all this into a Pandas dataframe.

In [17]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,Richard L. Hearn Thermal Generating Station (c...,43.646694,-79.336638,-5.312209e+06,1.050219e+07,5992.495307
1,None,43.650419,-79.337200,-5.311609e+06,1.050219e+07,5840.376700
2,"Circle K, Carlaw Avenue, Leslieville, East Yor...",43.654145,-79.337761,-5.311009e+06,1.050219e+07,5747.173218
3,"99, Carlaw Avenue, Leslieville, East York, Tor...",43.657871,-79.338323,-5.310409e+06,1.050219e+07,5715.767665
4,None,43.661597,-79.338885,-5.309809e+06,1.050219e+07,5747.173218
5,"1301, Dundas Street East, Leslieville, East Yo...",43.665323,-79.339447,-5.309209e+06,1.050219e+07,5840.376700
6,"Food Basics, 1000, Gerrard Street East, East E...",43.669050,-79.340009,-5.308609e+06,1.050219e+07,5992.495307
7,"Regatta Road, Port Lands, Toronto—Danforth, Ol...",43.640753,-79.340238,-5.313109e+06,1.050271e+07,5855.766389
8,"Ship Channel, Basin Street, Pinewood Toronto S...",43.644478,-79.340800,-5.312509e+06,1.050271e+07,5604.462508
9,None,43.648203,-79.341362,-5.311909e+06,1.050271e+07,5408.326913


### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on arena's in each neighborhood.

We're interested in venues in 'Athletics & Sports' category, but only those that are proper indoor arena's/gym's - soccer fields, tennis courts, hockey fields etc... are not direct competitors so we don't care about those. So we will include in out list only venues that have 'Courts' in category name.

Foursquare credentials are defined in hidden cell bellow.

In [19]:
foursquare_client_id =  # your Foursquare ID
foursquare_client_secret =  # your Foursquare Secret

In [20]:
# Category IDs corresponding to 'Courts' were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):
sports_category = '4f4528bc4b90abdf24c9de85' # 'Root' category for all Athletics & Sports venues

#Category of interest like Badminton courts , Volleyball courts
indoor_recreation_categories = ['52e81612bcbc57f1066b7a2b','4bf58dd8d48988d1e1941735','52e81612bcbc57f1066b7a2d',
                                '56aa371be4b08b9a8d57351a','52e81612bcbc57f1066b7a2f','4eb1bf013b7b6f98df247e07']

#Category for any indoor activity like Gym, Pilates Studio, Yoga Studio etc...
recreation_categories = ['52e81612bcbc57f1066b7a2e','4bf58dd8d48988d175941735','4bf58dd8d48988d102941735',
                         '52f2ab2ebcbc57f1066b8b47','503289d391d4c4b30a586d6a','52f2ab2ebcbc57f1066b8b49',
                         '4bf58dd8d48988d105941735','4bf58dd8d48988d176941735','52f2ab2ebcbc57f1066b8b48',
                         '4bf58dd8d48988d101941735','5744ccdfe4b0c0459246b4b2']

def is_indoorArena(categories, specific_filter=None, specific_filter2=None):
    indoor_words = ['court', 'club', 'gym', 'bowling', 'dojo', 'studio', 'sports']
    indoor = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in indoor_words:            
            if r in category_name:
                indoor = True
        if (('field') in category_name or ('park') in category_name or ('rink') in category_name or ('pitch') in category_name):
            indoor = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            indoor = True
        if not(specific_filter2 is None) and (category_id in specific_filter2):
            specific = False
            indoor = True
    return indoor, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Canada', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20200131'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except Exception as e: 
        traceback.print_exc()
        venues = []
    return venues

In [21]:
# Let's get the neighborhood locations of all found Gyms(all types) and all found indoor courts
def get_indoorArenas(lats, lons):
    arenas = {}
    indoor_arenas = {}
    location_arenas = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to make sure we have overlaps/full coverage so we don't miss any arena.
        venues = get_venues_near_location(lat, lon, sports_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_arenas = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_arena, is_indoor = is_indoorArena(venue_categories, specific_filter=indoor_recreation_categories, specific_filter2=recreation_categories)
            if is_arena:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                arena = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_indoor, x, y)
                if venue_distance<=300:
                    area_arenas.append(arena)
                arenas[venue_id] = arena
                if is_indoor:
                    indoor_arenas[venue_id] = arena
        location_arenas.append(area_arenas)
        print(' .', end='')
    print(' done.')
    return arenas, indoor_arenas, location_arenas

# Try to load from local file system in case we did this before
arenas = {}
indoor_arenas = {}
location_arenas = []
loaded = False
try:
    with open('arenas1_350.pkl', 'rb') as f:
        arenas = pickle.load(f)
    with open('indoor_arenas1_350.pkl', 'rb') as f:
        indoor_arenas = pickle.load(f)
    with open('location_arenas_350.pkl', 'rb') as f:
        location_arenas = pickle.load(f)
    print('Arena data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    arenas, indoor_arenas, location_arenas = get_indoorArenas(latitudes, longitudes)

        # Let's persists this in local file system
    with open('arenas1_350.pkl', 'wb') as f:
        pickle.dump(arenas, f)
    with open('indoor_arenas1_350.pkl', 'wb') as f:
        pickle.dump(indoor_arenas, f)
    with open('location_arenas_3501.pkl', 'wb') as f:
        pickle.dump(location_arenas, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [22]:
print('Total number of arenas/gyms:', len(arenas))
print('Total number of indoor sports courts:', len(indoor_arenas))
print('Percentage of indoor sports courts: {:.2f}%'.format(len(indoor_arenas) / len(arenas) * 100))
print('Average number of sports courts in neighborhood:', np.array([len(r) for r in location_arenas]).mean())

Total number of arenas/gyms: 527
Total number of indoor sports courts: 11
Percentage of indoor sports courts: 2.09%
Average number of sports courts in neighborhood: 2.39010989010989


In [23]:
print('List of all Arenas')
print('-----------------------')
for r in list(arenas.values())[:10]:
    print(r)
print('...')
print('Total:', len(arenas))

List of all Arenas
-----------------------
('504b5c47e4b0b18004b3b97c', 'Urban Warrior Toronto', 43.64548621977247, -79.33439048159343, '440 Unwin, Toronto ON', 225, False, -5312429.46078515, 10501953.479111955)
('5034cd9de4b04e46b78a7038', 'Mayfair Racquet and Fitness Clubs', 43.65370736853231, -79.33874219198448, '801 Lake Shore Blvd. E, Toronto ON', 323, False, -5311066.027554018, 10502313.175787086)
('52032a737dd26ccfe22fbd40', 'Truelifefitness', 43.65591886339037, -79.33774709701538, '11 Carlaw Avenue Unit # 1, Toronto ON M4M 2R6', 197, False, -5310726.690955734, 10502159.672581958)
('52dbbc9b498e8398e2dafe33', 'Fortis Fitness', 43.65605217441838, -79.33747355564634, '11 Carlaw Ave (Carlaw and Lakeshore), Toronto ON M4M 2R6', 213, False, -5310708.936450162, 10502125.74628907)
('52c9880b498edccef881e7b0', 'Dwell Gym', 43.6634116374049, -79.3411044597478, 'Carlaw Ave (Carlaw & Dundas), Toronto Ont', 204, False, -5309491.9737533, 10502416.96993303)
('582a5145eaf46a4e3237ce99', '9Roun

In [24]:
print('List of Indoor Arenas')
print('---------------------------')
for r in list(indoor_arenas.values())[:12]:
    print(r)
print('...')
print('Total:', len(indoor_arenas))

List of Indoor Arenas
---------------------------
('4af5b6a9f964a520c7fb21e3', 'Royal Canadian Curling Club', 43.65998933676023, -79.34955038817158, '131 Broadview Avenue (at Queen St), Toronto ON M4M 2E9', 241, True, -5309929.526822429, 10503452.270574277)
('57dd8898498e8b15625c4da0', 'Swatch Major Series', 43.641736, -79.354811, '11 Polson St, Toronto ON M5A 1A4', 336, True, -5312767.517354713, 10504379.196218764)
('4e46a5327d8b91a06591657e', 'Bleecker Basketball Court', 43.66877391581396, -79.37500577132985, 'Toronto ON', 288, True, -5308208.382787281, 10506238.33792338)
('4e8dfaff61afe70d47808fa1', 'Nikey Court', 43.670839237536654, -79.37343944281525, 'Canada', 187, True, -5307899.814817673, 10506021.2252819)
('5137771be4b0f5385c5941a4', 'Mattamy Athletic Centre Coca-Cola Court', 43.66185745169743, -79.37914878866015, '50 Carlton Street (Church and Carlton), Toronto ON', 340, True, -5309255.806874246, 10506838.29473631)
('4fdc78cbe4b044d93b326197', 'HOPE Volleyball Tournament', 43

In [25]:
print('Arenas around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_arenas[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Arenas around location {}: {}'.format(i+1, names))

Arenas around location
---------------------------
Arenas around location 101: 
Arenas around location 102: Corus Studio-Bootcamp, Homewood Gym (condo-private)
Arenas around location 103: The Market Club, Socially Fit
Arenas around location 104: St. Lawrence Community Centre, Scullhouse Rowing, Kings Court Gym, St Lawrence Community Centre Pool
Arenas around location 105: Wilson's, 9Round Fitness, Space Lofts, Fuel East, Richmond Gym, Kings Court Gym, John Innes Community Centre, Moss Park Tennis Courts
Arenas around location 106: John Innes Community Centre, Moss Park Tennis Courts
Arenas around location 107: Smush Room, GoodLife Fitness 7 Station Street Toronto, Aikido Tendoka, Renaissance Yoga & Ayurveda
Arenas around location 108: Eagle Tae Kwon Do, Cabbagetown Boxing Club, Martin Hung Karate, Sundara Yoga Studio
Arenas around location 109: Cabbagetown Boxing Club, Bleecker Basketball Court, Toronto Racquet Club
Arenas around location 110: Nikey Court


Let's now see all the collected arenas in our area of interest on map, and let's also show indoor arenas in different color.

In [26]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.Marker([latitude, longitude], popup='DownTown').add_to(map_toronto)
for res in arenas.values():
    lat = res[2]; lon = res[3]
    is_indoor = res[6]
    color = 'red' if is_indoor else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_toronto)
map_toronto


So now we have all the arenas in area within few kilometers from Downtown Toronto, and we know which ones are Indoor sports courts! We also know which arenas exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new indoor arena!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Toronto that have low arena/gym density, particularly those with low number of indoor multi-court arena. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every arena within 6km from Toronto center** (Downtown Toronto). We have also **identified indoor courts ** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**arena density**' across different areas of Toronto - we will use **heatmaps** to identify a few promising areas close to center with low number of arena/gym in general (*and* no indoor sports arena in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two gyms/arenas in radius of 250 meters**, and we want locations **without indoor arena in radius of 1000 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of arenas in every area candidate**:

In [94]:
location_arenas_count = [len(arena) for arena in location_arenas]

df_locations['Arena in area'] = location_arenas_count

print('Average number of arenas/gym in every area with radius=300m:', np.array(location_arenas_count).mean())

df_locations.head(11)

Average number of arenas/gym in every area with radius=300m: 2.39010989010989


,Address,Latitude,Longitude,X,Y,Distance from center,Arena in area,Distance to Indoor arena
0,Richard L. Hearn Thermal Generating Station (c...,43.646694,-79.336638,-5.312209e+06,1.050219e+07,5992.495307,1,2257.195055
1,None,43.650419,-79.337200,-5.311609e+06,1.050219e+07,5840.376700,0,2099.457368
2,"Circle K, Carlaw Avenue, Leslieville, East Yor...",43.654145,-79.337761,-5.311009e+06,1.050219e+07,5747.173218,3,1659.085726
3,"99, Carlaw Avenue, Leslieville, East York, Tor...",43.657871,-79.338323,-5.310409e+06,1.050219e+07,5715.767665,1,1348.113369
4,None,43.661597,-79.338885,-5.309809e+06,1.050219e+07,5747.173218,8,1265.801669
5,"1301, Dundas Street East, Leslieville, East Yo...",43.665323,-79.339447,-5.309209e+06,1.050219e+07,5840.376700,11,1451.584677
6,"Food Basics, 1000, Gerrard Street East, East E...",43.669050,-79.340009,-5.308609e+06,1.050219e+07,5992.495307,1,1825.360864
7,"Regatta Road, Port Lands, Toronto—Danforth, Ol...",43.640753,-79.340238,-5.313109e+06,1.050271e+07,5855.766389,1,1701.918208
8,"Ship Channel, Basin Street, Pinewood Toronto S...",43.644478,-79.340800,-5.312509e+06,1.050271e+07,5604.462508,0,1687.293228
9,None,43.648203,-79.341362,-5.311909e+06,1.050271e+07,5408.326913,0,1875.470950


OK, now let's calculate the **distance to nearest Indoor arena from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [95]:
distances_to_indoor_arena = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in indoor_arenas.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_indoor_arena.append(min_distance)

df_locations['Distance to Indoor arena'] = distances_to_indoor_arena

In [96]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Arena in area,Distance to Indoor arena
0,Richard L. Hearn Thermal Generating Station (c...,43.646694,-79.336638,-5.312209e+06,1.050219e+07,5992.495307,1,2257.195055
1,None,43.650419,-79.337200,-5.311609e+06,1.050219e+07,5840.376700,0,2099.457368
2,"Circle K, Carlaw Avenue, Leslieville, East Yor...",43.654145,-79.337761,-5.311009e+06,1.050219e+07,5747.173218,3,1659.085726
3,"99, Carlaw Avenue, Leslieville, East York, Tor...",43.657871,-79.338323,-5.310409e+06,1.050219e+07,5715.767665,1,1348.113369
4,None,43.661597,-79.338885,-5.309809e+06,1.050219e+07,5747.173218,8,1265.801669
5,"1301, Dundas Street East, Leslieville, East Yo...",43.665323,-79.339447,-5.309209e+06,1.050219e+07,5840.376700,11,1451.584677
6,"Food Basics, 1000, Gerrard Street East, East E...",43.669050,-79.340009,-5.308609e+06,1.050219e+07,5992.495307,1,1825.360864
7,"Regatta Road, Port Lands, Toronto—Danforth, Ol...",43.640753,-79.340238,-5.313109e+06,1.050271e+07,5855.766389,1,1701.918208
8,"Ship Channel, Basin Street, Pinewood Toronto S...",43.644478,-79.340800,-5.312509e+06,1.050271e+07,5604.462508,0,1687.293228
9,None,43.648203,-79.341362,-5.311909e+06,1.050271e+07,5408.326913,0,1875.470950


In [97]:
print('Average distance to closest Indoor arena from each area center:', df_locations['Distance to Indoor arena'].mean())

Average distance to closest Indoor arena from each area center: 1495.5170853159175


OK, so **on average Indoor arena can be found within ~1500m** from every area center candidate.

Let's crete a map showing **heatmap / density of sports arenas** and try to extract some meaningfull info from that. 

In [98]:
arenas_latlons = [[res[2], res[3]] for res in arenas.values()]
indoor_latlons = [[res[2], res[3]] for res in indoor_arenas.values()]

In [99]:
from folium import plugins
from folium.plugins import HeatMap

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_toronto) #cartodbpositron cartodbdark_matter
HeatMap(arenas_latlons).add_to(map_toronto)
folium.Marker([latitude, longitude]).add_to(map_toronto)
folium.Circle([latitude, longitude], radius=1000, fill=False, color='white').add_to(map_toronto)
folium.Circle([latitude, longitude], radius=2000, fill=False, color='white').add_to(map_toronto)
folium.Circle([latitude, longitude], radius=3000, fill=False, color='white').add_to(map_toronto)
map_toronto

Looks like a few pockets of low arena/gym density closest to city center can be found **north, north-west from DownTown Toronto**. 

Let's create another heatmap map showing **heatmap/density of Indoor sports courts** only.

In [100]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_toronto) #cartodbpositron cartodbdark_matter
HeatMap(indoor_latlons).add_to(map_toronto)
folium.Marker([latitude, longitude]).add_to(map_toronto)
folium.Circle([latitude, longitude], radius=1000, fill=False, color='white').add_to(map_toronto)
folium.Circle([latitude, longitude], radius=2000, fill=False, color='white').add_to(map_toronto)
folium.Circle([latitude, longitude], radius=3000, fill=False, color='white').add_to(map_toronto)
map_toronto

Based on this we will now focus our analysis on areas *north, north-west from DownTown Toronto* - we will move the center of our area of interest and reduce it's size to have a radius of **2.5km**. This places our location candidates mostly in boroughs **West Toronto** 

Let's define new, more narrow region of interest, which will include low-arena-count parts of West Toronto closest to Downtown Toronto.

In [101]:
roi_x_min = toronto_center_x + 500
roi_y_max = toronto_center_y + 3500
roi_width = 2500
roi_height = 2500
roi_center_x = roi_x_min + 1500
roi_center_y = roi_y_max - 1500
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_toronto = folium.Map(location=roi_center, zoom_start=14)
HeatMap(arenas_latlons).add_to(map_toronto)
folium.Marker([latitude, longitude]).add_to(map_toronto)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_toronto)
map_toronto

This nicely covers all the pockets of low arena/gym density in West Toronto closest to Downtown center.

Let's create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 100m appart).

In [102]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

1953 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of gyms/arenas in vicinity** (we'll use radius of **500 meters**) and **distance to closest indoor sports arena**.

In [103]:
def count_areans_nearby(x, y, arenas, radius=500):    
    count = 0
    for res in arenas.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_arenas(x, y, arenas):
    d_min = 100000
    for res in arenas.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_arenas_counts = []
roi_indoor_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_areans_nearby(x, y, arenas, radius=500)
    roi_arenas_counts.append(count)
    distance = find_nearest_arenas(x, y, indoor_arenas)
    roi_indoor_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [104]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Arenas nearby':roi_arenas_counts,
                                 'Distance to Indoor arenas':roi_indoor_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Arenas nearby,Distance to Indoor arenas
0,43.666416,-79.384776,-5.308459e+06,1.050741e+07,14,979.669451
1,43.667037,-79.384871,-5.308359e+06,1.050741e+07,14,1062.614203
2,43.662941,-79.384997,-5.309009e+06,1.050749e+07,14,701.246731
3,43.663562,-79.385091,-5.308909e+06,1.050749e+07,14,742.390523
4,43.664183,-79.385186,-5.308809e+06,1.050749e+07,13,794.065741
5,43.664804,-79.385281,-5.308709e+06,1.050749e+07,13,854.363572
6,43.665425,-79.385375,-5.308609e+06,1.050749e+07,13,921.593091
7,43.666046,-79.385470,-5.308509e+06,1.050749e+07,15,994.349303
8,43.666667,-79.385565,-5.308409e+06,1.050749e+07,15,1071.506999
9,43.667288,-79.385659,-5.308309e+06,1.050749e+07,18,1152.182260


OK. Let us now **filter** those locations: we're interested only in **locations with no more than two arena/gym in radius of 500 meters**, and **no indoor sports court arena in radius of 2000 meters**.

In [105]:
good_arenas_count = np.array((df_roi_locations['Arenas nearby']<=2))
print('Locations with no more than two arenas nearby:', good_arenas_count.sum())

good_court_distance = np.array(df_roi_locations['Distance to Indoor arenas']>=2000)
print('Locations with no indoor sports courts within 1000m:', good_court_distance.sum())

good_locations = np.logical_and(good_arenas_count, good_court_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two arenas nearby: 534
Locations with no indoor sports courts within 1000m: 353
Locations with both conditions met: 39


Let's see how this looks on a map.

In [106]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_toronto = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_toronto)
HeatMap(arenas_latlons).add_to(map_toronto)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_toronto)
folium.Marker([latitude, longitude]).add_to(map_toronto)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_toronto) 
map_toronto

We now have a bunch of locations fairly close to DownTown Toronto.Let's now show those good locations in a form of heatmap:

In [107]:
map_toronto = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_toronto)
folium.Marker([latitude, longitude]).add_to(map_toronto)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_toronto)
map_toronto

We now have a clear indication of zones with low number of gyms in vicinity, and *no* Indoor sports arena at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [108]:
from sklearn.cluster import KMeans

number_of_clusters = 3

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_toronto = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_toronto)
HeatMap(arenas_latlons).add_to(map_toronto)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_toronto)
folium.Marker([latitude, longitude]).add_to(map_toronto)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_toronto) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_toronto)
map_toronto

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [109]:
map_toronto = folium.Map(location=roi_center, zoom_start=14)
folium.Marker([latitude, longitude]).add_to(map_toronto)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=500, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_toronto)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_toronto)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_toronto) 
map_toronto

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [110]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addres = get_address(lat, lon)
    address = str(addres)
    address = address.replace(', Canada', '')
    candidate_area_addresses.append(address)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, toronto_center_x, toronto_center_y)
    print('{}{} => {:.1f}km from DownTown'.format(address, ' '*(50-len(address)), d/1000))
    

Addresses of centers of areas recommended for further analysis

Varsity Centre, Devonshire Place, Bloor Street Culture Corridor, University—Rosedale, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5S 1V6 => 2.3km from DownTown
None                                               => 4.4km from DownTown
366, Brunswick Avenue, The Annex, University—Rosedale, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5S 1X5 => 3.4km from DownTown


This concludes our analysis. We have created 3 addresses representing centers of zones containing locations with low number of gyms or arena and no indoor sports courts nearby, all zones being fairly close to city center (all less than 5km from DownTown, and about half of those around 3km from DownTown). 

In [78]:
map_toronto = folium.Map(location=roi_center, zoom_start=14)
folium.Circle([latitude, longitude], radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_toronto)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]]).add_to(map_toronto) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_toronto)
map_toronto

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of arenas or gyms in Downtown Toronto (~525 in our initial area of interest which was 6x6km around DownTown Toronto), there are pockets of low gym/arena density relatively close to city center. Highest concentration of gym/arena was detected south and east from Downtown Toronto, so we focused our attention to areas north, north-west and west, corresponding to boroughs West Toronto.

After directing our attention to this more narrow area of interest we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two gym/arena in radius of 500m and those with an Indoor sports arena closer than 2000m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 3 zones containing largest number of potential new sports arena locations based on number of and distance to existing venues. This, of course, does not imply that those zones are actually optimal locations for a new arena! Purpose of this analysis was to only provide info on areas close to Toronto center but not crowded with existing gyms or sports arena . It is entirely possible that there is a very good reason for small number of gyms or arenas in any of those areas, reasons which would make them unsuitable for a new sports arena regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Toronto areas close to center with low number of gyms and sports arenas in order to aid stakeholders in narrowing down the search for optimal location for a new multi-sport recreational sports court. By calculating gyms and sports arena density distribution from Foursquare data we have first identified general boroughs that justify further analysis (West Toronto), and then generated extensive collection of locations which satisfy some basic requirements. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like projected population boom, real estate development etc...